In [39]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [40]:
# Configure gmaps to use Google API key.
gmaps.configure(api_key=g_key)

In [31]:
# Store the WeatherPy_challenge.csv into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Weather,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Rain,Snow
0,0,Zunheboto,IN,2020-02-23 03:09:50,25.97,94.52,Rain,light rain,66.76,57,100,0.47,0.31,0.0
1,1,Innisfail,AU,2020-02-23 03:09:50,-17.53,146.03,Rain,moderate rain,83.52,83,100,5.03,3.13,0.0
2,2,Tuban,ID,2020-02-23 03:09:51,-6.90,112.06,Rain,light rain,84.27,77,54,8.03,1.13,0.0
3,3,Cape Town,ZA,2020-02-23 03:02:40,-33.93,18.42,Clear,clear sky,64.99,72,0,6.93,0.00,0.0
4,4,Carnarvon,AU,2020-02-23 03:05:52,-24.87,113.63,Clear,clear sky,82.40,58,0,8.05,0.00,0.0


In [32]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_yn = input("Do you want it to be raining? (yes/no) ")
snow_yn = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [33]:
# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]

# Filter for Rain criteria.
if rain_yn == "yes":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain"] == 0)]

# Filter for Snow criteria.
if snow_yn == "yes":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow"] == 0)]    

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Weather,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Rain,Snow
4,4,Carnarvon,AU,2020-02-23 03:05:52,-24.87,113.63,Clear,clear sky,82.40,58,0,8.05,0.0,0.0
5,5,Rikitea,PF,2020-02-23 03:05:35,-23.12,-134.97,Clouds,few clouds,79.86,66,12,6.15,0.0,0.0
11,11,Lorengau,PG,2020-02-23 03:07:10,-2.02,147.27,Clear,clear sky,83.62,67,0,3.71,0.0,0.0
19,19,Puerto Escondido,MX,2020-02-23 03:09:54,15.85,-97.07,Clear,clear sky,82.40,88,5,6.93,0.0,0.0
21,21,Tamboril,DO,2020-02-23 03:09:54,19.49,-70.61,Clouds,broken clouds,77.00,100,75,4.70,0.0,0.0
23,23,Vaini,TO,2020-02-23 03:05:53,-21.20,-175.20,Clouds,scattered clouds,84.20,79,40,13.87,0.0,0.0
30,30,Bireun,ID,2020-02-23 03:09:57,5.20,96.70,Clouds,few clouds,75.90,73,19,3.06,0.0,0.0
32,32,Busselton,AU,2020-02-23 03:00:48,-33.65,115.33,Clouds,overcast clouds,84.99,33,99,10.38,0.0,0.0
44,44,Namibe,AO,2020-02-23 03:10:00,-15.20,12.15,Clouds,broken clouds,75.56,77,84,12.39,0.0,0.0
47,47,Pundaguitan,PH,2020-02-23 03:08:38,6.37,126.17,Clear,clear sky,80.58,76,7,9.13,0.0,0.0


In [34]:
# Create DataFrame called hotel_df to store hotel names along with city data.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,82.40,clear sky,-24.87,113.63,
5,Rikitea,PF,79.86,few clouds,-23.12,-134.97,
11,Lorengau,PG,83.62,clear sky,-2.02,147.27,
19,Puerto Escondido,MX,82.40,clear sky,15.85,-97.07,
21,Tamboril,DO,77.00,broken clouds,19.49,-70.61,
23,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,
30,Bireun,ID,75.90,few clouds,5.20,96.70,
32,Busselton,AU,84.99,overcast clouds,-33.65,115.33,
44,Namibe,AO,75.56,broken clouds,-15.20,12.15,
47,Pundaguitan,PH,80.58,clear sky,6.37,126.17,


In [35]:
hotel_df.count()

City                   120
Country                120
Max Temp               120
Weather Description    120
Lat                    120
Lng                    120
Hotel Name             120
dtype: int64

### Retrieve Hotels from a Nearby Search

In [36]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [37]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,82.40,clear sky,-24.87,113.63,Hospitality Carnarvon
5,Rikitea,PF,79.86,few clouds,-23.12,-134.97,Pension Maro'i
11,Lorengau,PG,83.62,clear sky,-2.02,147.27,Lorengau Harbourside Hotel
19,Puerto Escondido,MX,82.40,clear sky,15.85,-97.07,Aldea del Bazar Hotel and Spa
21,Tamboril,DO,77.00,broken clouds,19.49,-70.61,Camp David Ranch Restaurant
23,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
30,Bireun,ID,75.90,few clouds,5.20,96.70,OYO 2303 Hotel Blang Raya
32,Busselton,AU,84.99,overcast clouds,-33.65,115.33,Observatory Guest House
44,Namibe,AO,75.56,broken clouds,-15.20,12.15,Chik-Chik Namibe
47,Pundaguitan,PH,80.58,clear sky,6.37,126.17,Leonardo Family Beach House 1


In [49]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_vacation.csv"
# Export the Hotel_df into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))